In [ ]:
# code adapted from https://github.com/karpathy/ng-video-lecture

import torch 
import torch.nn as nn 
from torch.nn import functional as F 

In [ ]:
# hyperparameters
batch_size = 32 
block_size = 8 
max_iters = 3000
eval_interval = 300
learning_rate = 1e-2

device = 'cuda' if torch.cuda.is_available() else "cpu"
eval_iters = 200

In [ ]:
torch.manual_seed(1337)

In [ ]:
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [ ]:
with open("input.txt", "r", encoding="utf-8") as f:
    text = f.read()

In [ ]:
# get the unique characters in the text

chars = sorted(list(set(text)))
vocab_size = len(chars)

In [ ]:
# create a mapping from characters to integers and vice-versa

stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s] # string to list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # integers to string

In [ ]:
# train and test splits

data = torch.tensor(encode(text), dtype=torch.long)

In [ ]:
n = int(.9*len(data)) # reserve 90% for training and 10% for validation
train_data = data[:n]
val_data = data[n:]

In [ ]:
# data loading 

def get_batch(split):
    # create a small batch of data of inputs x and targets y
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data)-block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [ ]:
class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from the lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)

        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    

    def generate(self, idx, max_new_tokens):
        # idx is (B,T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)

            # focus only on the last time step 
            logits = logits[:, -1, :] # (B,C)
            # apply softmax to get the probabilites
            probs = F.softmax(logits, dim=-1) # (B,C)

            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B,1)
            # append the sampled index to the running sequence
            idx = torch.cat([idx, idx_next], dim=1) # (B,T+1)
        return idx

In [ ]:
model = BigramLanguageModel()

model = model.to(device)

In [ ]:
# out = model.generate(torch.tensor([[7]]),5)

In [ ]:
# [decode(x.tolist()) for x in out]

In [ ]:
@torch.no_grad()
def evaluate_loss():
    out = {}
    model.eval()
    for split in ["train", "val"]:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [ ]:
for it in range(10000):
    if (it % eval_iters) == 0:
        losses = evaluate_loss()
        print(f"step {it}: train loss {losses['train']:.4f} val loss {losses['val']:.4f}")

    xb, yb = get_batch('train')

    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

context = torch.zeros((1,1), dtype=torch.long, device=device)

print(decode(model.generate(context, max_new_tokens=500)[0].tolist()))
